# Модель SARIMA (пакет [`sktime`](https://www.sktime.net/en/stable/))

In [ ]:
import numpy as np
import pandas as pd

from sktime.forecasting.arima import AutoARIMA # из pmdarima
from sktime.forecasting.arima import StatsModelsARIMA as ARIMA 
from sktime.forecasting.sarimax import SARIMAX # альтернатива из statsmodels
from sktime.utils.plotting import plot_series

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) дневные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`DGS10`](https://fred.stlouisfed.org/series/DGS10)) с 2010-01-01 по 2023-12-31 и создадим датафрейм `y`

In [ ]:
y = web.DataReader(name='DGS10', data_source='fred', start='2010-01-01', end='2023-12-31').dropna()
plot_series(y)

## Подгонка SARIMA заданного порядка без сноса

Подгоним модель ARMA(1,1,1)(1,1,1,7) без сноса для `y`

Спецификация

$$
	(1-\phi L)(1-\Phi L^7)(1-L)(1-L^7) y_t=(1+\theta L)(1+\Theta L^7)u_t
$$

In [ ]:
forecaster = ARIMA(order=(1,1,1), seasonal_order=(1,1,1,7), trend='n')
forecaster.fit(y)
forecaster.summary()

In [ ]:
forecaster = SARIMAX(order=(1,1,1), seasonal_order=(1,1,1,7), trend='n')
forecaster.fit(y)
forecaster.summary()

# Автоматический выбор порядка модели

Выбор порядка по информационным критериям. Выведем только порядок подогнанной модели

In [ ]:
forecaster = AutoARIMA(information_criterion='aic', seasonal=True, sp=7)
forecaster.fit(y)
forecaster.get_fitted_params() #['order']

In [ ]:
forecaster = AutoARIMA(information_criterion='bic', seasonal=True, sp=7)
forecaster.fit(y)
forecaster.get_fitted_params() #['order']